<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.1


    Uninstalling protobuf-6.33.1:
      Successfully uninstalled protobuf-6.33.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:


      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-11-24 15:13:53
-------------------
qualified stocks: 87
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  400.00
Current:  1.42 C
-------------------
Today PnL: -79.69 K (-0.56%)
Current PnL: -23.40 L (-15.33%)
CY Booked + Current PnL: -9.51 L (-6.23%)
-------------------
Total profit:  1.38 L
Total loss:  -24.77 L
-------------------
Total Booked + Current PnL: 17.47 L (13.91%)
Total Booked PnL: 40.87 L (32.53%)
Curr Year Booked PnL: 13.89 L (9.75%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.31 C
Est FTT PnL: 88.81 L (62.37%)
Deployed:  1.26 C
Current:  1.42 C
CAGR/XIRR %: 7.14%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,TTKPRESTIG,770.00,0.03,-14.21,16.67,0.09,14413.0,-14317.0,86460.0,100.64,49.0,M-SC,4.25,253.0,-0.99,0.61,13.59,OX40N,NTT,DURABLES
39,INDIGOPNTS,1408.00,-0.57,-9.50,10.52,0.02,16618.0,-16589.0,157970.0,141.07,69.0,M-SC,5.41,234.0,-1.00,1.11,37.41,OX40N,NTT,PAINTS
43,ITC,452.00,-1.20,-1.55,12.17,10.44,23980.0,-3095.0,197043.0,-41.15,42.0,X-LC,0.78,5.0,-0.13,1.38,3.93,X40,NTT,FMCG
35,ICICIGI,2252.93,-0.57,6.31,13.06,20.19,24202.0,10992.0,185312.0,-16.73,49.0,X-MC,7.62,68.0,0.45,1.30,22.37,X40,ATH,INSURANCE
53,NESTLEIND,1377.00,-0.78,12.03,8.37,21.40,24888.0,31918.0,297344.0,3.91,56.0,X-LC,7.90,12.0,1.28,2.09,19.47,XY25,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ACC,3906.00,4.20,-19.86,104.82,64.15,199892.0,-47251.0,190700.0,-53.25,64.0,X-SC,1.68,82.0,-0.24,1.34,6.97,XY24,BTT,CEMENT
14,CAMPUS,393.00,2.68,-9.14,46.70,33.28,71187.0,-15340.0,152435.0,-26.79,45.0,M-SC,2.95,222.0,-0.22,1.07,18.15,XY24,NTT,FOOTWEAR
29,HAPPSTMNDS,1480.71,2.56,-36.38,190.56,84.85,155375.0,-46629.0,81536.0,-24.31,51.0,H-SC,14.22,147.0,-0.30,0.57,4.65,AR,ATH,IT
57,RAJESHEXPO,518.00,2.47,-63.16,171.60,0.07,86728.0,-86636.0,50541.0,1752.27,60.0,L-SC,1.78,268.0,-1.00,0.35,25.70,OX40N,NTT,JEWELLERY
26,GILLETTE,11206.78,2.10,-0.16,32.62,32.42,82692.0,-396.0,253500.0,-17.85,38.0,X-SC,6.62,85.0,-0.00,1.78,14.92,X40,ATH,FMCG


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
81,VALIANTORG,838.00,-4.99,-46.96,228.37,74.15,198152.0,-76837.0,86768.0,-255.96,24.0,H-SC,14.64,149.0,-0.39,0.61,11.22,XR,NTT,CHEMICALS
62,ROUTE,2227.21,-4.56,-51.47,248.36,69.05,160376.0,-68492.0,64574.0,-60.85,26.0,H-SC,23.42,140.0,-0.43,0.45,0.00,SR,ATH,IT
65,SHALBY,327.00,-3.86,-20.11,56.59,25.10,81896.0,-36424.0,144719.0,963.80,27.0,M-SC,16.43,250.0,-0.44,1.02,15.98,XY24,NTT,HEALTHCARE
58,RAJOOENG,143.10,-3.74,-30.27,100.22,39.61,71627.0,-31030.0,71470.0,-52.76,20.0,H-SC,19.30,135.0,-0.43,0.50,0.00,AR,ATH,MISC
72,TANLA,1943.92,-3.69,-37.13,249.44,119.69,424617.0,-100533.0,170228.0,-36.34,26.0,H-SC,12.00,156.0,-0.24,1.20,34.48,AR,ATH,IT


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
38,INDIAMART,4810.62,-1.54,-0.92,108.63,106.72,132753.0,-1129.0,122207.0,-51.01,32.0,H-SC,4.05,139.0,-0.01,0.86,22.25,AR,ATH,MISC


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-1.35,-5.27,119.49,107.91,167115.0,-7787.0,139857.0,-22.98,32.0,M-SC,10.37,216.0,-0.05,0.98,0.00,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,-0.57,-9.50,10.52,0.02,16618.0,-16589.0,157970.0,141.07,69.0,M-SC,5.41,234.0,-1.00,1.11,37.41,OX40N,NTT,PAINTS
78,TTKPRESTIG,770.00,0.03,-14.21,16.67,0.09,14413.0,-14317.0,86460.0,100.64,49.0,M-SC,4.25,253.0,-0.99,0.61,13.59,OX40N,NTT,DURABLES
47,KANSAINER,340.00,-0.34,-22.57,46.55,13.47,97196.0,-60867.0,208800.0,-68.12,27.0,H-SC,1.52,158.0,-0.63,1.47,6.17,XY24,NTT,PAINTS
44,JCHAC,2282.00,0.02,-23.23,30.25,-0.01,29679.0,-29693.0,98112.0,19986.96,63.0,M-SC,1.59,233.0,-1.00,0.69,11.10,OX40N,NTT,AC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.0,1.05,1.33,70.01,72.27,120207.0,2252.0,171700.0,-12.89,56.0,M-LC,4.62,99.0,0.02,1.21,7.59,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.00,1.05,1.33,70.01,72.27,120207.0,2252.0,171700.0,-12.89,56.0,M-LC,4.62,99.0,0.02,1.21,7.59,XR,NTT,IT
38,INDIAMART,4810.62,-1.54,-0.92,108.63,106.72,132753.0,-1129.0,122207.0,-51.01,32.0,H-SC,4.05,139.0,-0.01,0.86,22.25,AR,ATH,MISC
25,FINCABLES,1641.55,-1.35,-5.27,119.49,107.91,167115.0,-7787.0,139857.0,-22.98,32.0,M-SC,10.37,216.0,-0.05,0.98,0.00,OX40N,ATH,CABLES
86,ZYDUSLIFE,1286.17,0.24,-2.30,38.82,35.63,79486.0,-4818.0,204756.0,-15.85,33.0,H-MC,4.52,119.0,-0.06,1.44,13.99,AR,ATH,PHARMA
84,WHIRLPOOL,2270.00,0.22,-5.20,91.90,81.92,100012.0,-5971.0,108827.0,-44.42,32.0,M-SC,5.33,236.0,-0.06,0.76,28.83,XR,NTT,DURABLES


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
34,HONAUT,58357.33,-1.59,-16.17,70.96,43.32,96889.0,-26332.0,136540.0,-27.94,22.0,X-SC,1.72,90.0,-0.27,0.96,4.84,X40N,ATH,ELECTRICAL
76,TMPV,600.00,-1.15,-15.14,67.55,42.18,158443.0,-41854.0,234556.0,-24.30,22.0,X-LC,0.42,3.0,-0.26,1.65,0.00,XY24,NTT,AUTO
46,JSWINFRA,342.00,-0.26,-9.30,27.04,15.23,49134.0,-18637.0,181710.0,-22.32,24.0,X-MC,6.49,66.0,-0.38,1.28,19.95,X40N,NTT,REALTY
11,BATAINDIA,2096.00,0.53,-35.13,108.41,35.20,90493.0,-45197.0,83473.0,9.46,25.0,X-SC,12.61,92.0,-0.50,0.59,0.53,X40,NTT,FOOTWEAR
60,RELAXO,1176.00,-1.41,-47.85,192.03,52.31,145382.0,-69452.0,75708.0,-44.44,27.0,X-SC,3.81,91.0,-0.48,0.53,1.34,X40N,NTT,FOOTWEAR


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TMPV,600.00,-1.15,-15.14,67.55,42.18,158443.0,-41854.0,234556.0,-24.30,22.0,X-LC,0.42,3.0,-0.26,1.65,0.00,XY24,NTT,AUTO
43,ITC,452.00,-1.20,-1.55,12.17,10.44,23980.0,-3095.0,197043.0,-41.15,42.0,X-LC,0.78,5.0,-0.13,1.38,3.93,X40,NTT,FMCG
36,ICICIPRULI,790.00,-0.52,0.75,29.99,30.97,53403.0,1333.0,178071.0,-22.98,47.0,X-MC,1.52,75.0,0.02,1.25,13.41,X40,ATH,INSURANCE
3,ACC,3906.00,4.20,-19.86,104.82,64.15,199892.0,-47251.0,190700.0,-53.25,64.0,X-SC,1.68,82.0,-0.24,1.34,6.97,XY24,BTT,CEMENT
34,HONAUT,58357.33,-1.59,-16.17,70.96,43.32,96889.0,-26332.0,136540.0,-27.94,22.0,X-SC,1.72,90.0,-0.27,0.96,4.84,X40N,ATH,ELECTRICAL


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TMPV,600.00,-1.15,-15.14,67.55,42.18,158443.0,-41854.0,234556.0,-24.30,22.0,X-LC,0.42,3.0,-0.26,1.65,0.00,XY24,NTT,AUTO
9,BAJAJHFL,181.50,0.30,-13.42,71.47,48.45,124068.0,-26912.0,173594.0,-19.66,32.0,X-MC,7.46,64.0,-0.22,1.22,0.30,X40N,ATH,FINANCE
56,QUESS,424.00,0.38,-29.04,101.29,42.84,46725.0,-18876.0,46130.0,-53.72,37.0,X-SC,37.27,83.0,-0.40,0.32,0.38,XY24,NTT,MISC
11,BATAINDIA,2096.00,0.53,-35.13,108.41,35.20,90493.0,-45197.0,83473.0,9.46,25.0,X-SC,12.61,92.0,-0.50,0.59,0.53,X40,NTT,FOOTWEAR
54,PAGEIND,51605.08,-0.12,-5.12,32.87,26.07,51067.0,-8380.0,155360.0,-30.18,35.0,X-MC,8.87,55.0,-0.16,1.09,0.71,X40,ATH,APPARELS


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,QUESS,424.00,0.38,-29.04,101.29,42.84,46725.0,-18876.0,46130.0,-53.72,37.0,X-SC,37.27,83.0,-0.40,0.32,0.38,XY24,NTT,MISC
60,RELAXO,1176.00,-1.41,-47.85,192.03,52.31,145382.0,-69452.0,75708.0,-44.44,27.0,X-SC,3.81,91.0,-0.48,0.53,1.34,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,0.53,-35.13,108.41,35.20,90493.0,-45197.0,83473.0,9.46,25.0,X-SC,12.61,92.0,-0.50,0.59,0.53,X40,NTT,FOOTWEAR
22,DIXON,18931.72,-1.91,-5.73,28.97,21.58,38273.0,-8028.0,132111.0,-52.49,31.0,X-MC,7.05,56.0,-0.21,0.93,16.96,X40N,ATH,IT
52,MEDANTA,1486.00,1.51,3.99,19.32,24.08,25505.0,5062.0,132012.0,-4.36,47.0,X-SC,4.71,89.0,0.20,0.93,24.39,XY24,NTT,HEALTHCARE


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4389.97,-0.34,-14.69,39.82,19.28,117525.0,-50815.0,295141.0,-25.57,61.0,X-LC,7.64,1.0,-0.43,2.07,9.11,X40,ATH,IT
41,INFY,2275.00,0.13,5.31,47.06,54.87,155068.0,16623.0,329511.0,-18.16,60.0,X-LC,3.17,2.0,0.11,2.31,14.04,X40,BTT,IT
76,TMPV,600.00,-1.15,-15.14,67.55,42.18,158443.0,-41854.0,234556.0,-24.30,22.0,X-LC,0.42,3.0,-0.26,1.65,0.00,XY24,NTT,AUTO
82,VBL,671.64,-0.40,-9.94,50.64,35.67,144046.0,-31390.0,284452.0,-20.61,40.0,X-LC,6.06,4.0,-0.22,2.00,2.54,X40N,ATH,FMCG
43,ITC,452.00,-1.20,-1.55,12.17,10.44,23980.0,-3095.0,197043.0,-41.15,42.0,X-LC,0.78,5.0,-0.13,1.38,3.93,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,-0.90,-13.54,110.12,81.67,89317.0,-12701.0,81109.0,7294.44,49.0,L-SC,18.56,271.0,-0.14,0.57,57.07,XR,NTT,CERAMICS
68,SONACOMS,806.63,0.65,-12.40,59.29,39.54,52543.0,-12539.0,88620.0,-31.43,67.0,M-SC,6.83,220.0,-0.24,0.62,24.96,AR,ATH,AUTO
51,MASFIN,398.61,-2.54,-5.39,29.00,22.05,26883.0,-5280.0,92700.0,-18.25,49.0,H-SC,7.19,164.0,-0.20,0.65,34.75,XR,ATH,FINANCE
70,SURYODAY,216.00,-0.45,-20.02,53.84,23.03,77109.0,-35853.0,143218.0,55.70,48.0,H-SC,11.09,167.0,-0.46,1.01,42.10,XR,NTT,BANKS
39,INDIGOPNTS,1408.00,-0.57,-9.50,10.52,0.02,16618.0,-16589.0,157970.0,141.07,69.0,M-SC,5.41,234.0,-1.00,1.11,37.41,OX40N,NTT,PAINTS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,-0.57,-9.50,10.52,0.02,16618.0,-16589.0,157970.0,141.07,69.0,M-SC,5.41,234.0,-1.00,1.11,37.41,OX40N,NTT,PAINTS
5,ASIANPAINT,3460.25,0.13,0.68,20.13,20.95,51025.0,1707.0,253475.0,-0.35,76.0,X-LC,12.74,31.0,0.03,1.78,36.56,X40,ATH,PAINTS
40,INDUSINDBK,1800.00,-1.18,-38.24,115.17,32.89,54917.0,-29524.0,47683.0,-688.88,59.0,L-MC,5.57,259.0,-0.54,0.33,31.32,XR,NTT,BANKS
50,LTIM,7201.88,-0.20,8.12,21.78,31.66,57963.0,19979.0,266130.0,3.99,61.0,H-LC,12.46,43.0,0.34,1.87,47.66,X200,ATH,IT
0,5PAISA,593.00,-2.98,-36.67,78.27,12.90,108312.0,-80114.0,138382.0,104.33,55.0,H-SC,14.61,161.0,-0.74,0.97,15.06,OX40N,NTT,FINANCE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.83
1,25,44.72
2,50,75.55


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.54
MC    29.32
LC    25.13
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.01
X40      21.21
X40N     11.86
XR        9.61
XY25      9.08
AR        8.95
OX40N     7.67
X200      1.87
SR        0.96
MH        0.77
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    23.87
X-MC    22.89
X-LC    19.97
M-SC    12.01
X-SC     8.26
H-MC     4.72
H-LC     2.98
L-SC     1.40
M-MC     1.38
M-LC     1.21
L-LC     0.97
L-MC     0.33
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,16.83,-4.99,38.94
IT,13.78,-15.24,74.73
FINANCE,9.40,-15.74,64.64
MISC,7.37,-23.65,74.31
PAINTS,6.01,-8.01,24.05
ELECTRICAL,5.88,-11.99,53.01
INSURANCE,4.45,0.36,34.76
PHARMA,3.87,-1.36,33.80
AUTO,3.34,-20.98,72.86


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3049600.0,21
AR,1297113.0,10
XR,1295310.0,13
X40,1016216.0,14
X40N,761197.0,9
OX40N,697845.0,10
XY25,348970.0,6
SR,282601.0,2
MH,73990.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3578267.0,25
M-SC,1348646.0,15
X-MC,1223832.0,16
X-LC,918407.0,11
X-SC,744486.0,8
H-MC,407029.0,3
L-SC,261084.0,3
H-LC,133204.0,2
M-LC,120207.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1218408.0      6
           AR         881463.0      5
           XR         796543.0      7
M-SC       XY24       789570.0      6
X-MC       X40        446573.0      7
X-LC       X40        396458.0      5
X-MC       XY24       345683.0      3
H-SC       OX40N      325262.0      4
X-SC       X40N       299179.0      3
M-SC       OX40N      285855.0      5
H-SC       SR         282601.0      2
X-SC       XY24       272122.0      3
X-MC       X40N       247908.0      4
X-LC       XY24       233263.0      2
H-MC       AR         216475.0      2
X-LC       X40N       214110.0      2
H-MC       XY24       190554.0      1
X-MC       XY25       183668.0      2
L-SC       XR         174356.0      2
X-SC       X40        173185.0      2
M-SC       XR         149287.0      2
           AR         123934.0      2
M-LC       XR         120207.0      1
L-SC       OX40N       86728.0      1
H-LC       AR          75241.0      1
X-LC       XY25        74576.0      2
H-SC       MH          73990.0      1
H-LC       X200        57963.0      1
L-MC       XR          54917.0      1
M-MC       XY25        49202.0      1
L-LC       XY25        41524.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 39.0 seconds
